# Collect Data
For speed, we will use downsampled data.
You can use any gene fast file with this script.

In [1]:
!pip install biopython


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\dum\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\dum\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [3]:
! pip install numpy==1.23.4

  Using cached numpy-1.23.4.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "C:\Users\dum\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\dum\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\dum\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "C:\Users\dum\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_

In [4]:
! pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\dum\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [37]:
! pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\dum\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [33]:
# Import necessary libraries
from Bio import SeqIO
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Disable TensorFlow v2 behavior
tf.compat.v1.disable_v2_behavior()

# Enable plotting in a separate window
%matplotlib qt

# Function to convert FASTA file to DataFrame
def fasta2df(infile):
    records = SeqIO.parse(infile, 'fasta')
    seqList = []
    for record in records:
        desp = record.description
        seq = str(record.seq).upper()  # Convert the sequence directly to a string and make it uppercase
        seqList.append([desp] + [seq])
    seq_df = pd.DataFrame(seqList, columns=['strainName', 'seq'])
    return seq_df

# Load FASTA file and convert to DataFrame
df = fasta2df("data/alternative_splicing_human_10541.fasta")

# Display the first few rows of the DataFrame
df.head()


,strainName,seq
0,sp|A0A0K2S4Q6|CD3CH_HUMAN Protein CD300H OS=Ho...,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...
1,sp|A0A1B0GTW7|CIROP_HUMAN Ciliated left-right ...,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...
2,sp|A0AV02|S12A8_HUMAN Solute carrier family 12...,MTQMSQVQELFHEAAQQDALAQPQPWWKTQLFMWEPVLFGTWDGVF...
3,sp|A0AV96|RBM47_HUMAN RNA-binding protein 47 O...,MTAEDSTAAMSSDSAAGSSAKVPEGVAGAPNEAALLALMERTGYSM...
4,sp|A0AVF1|IFT56_HUMAN Intraflagellar transport...,MMLSRAKPAVGRGVQHTDKRKKKGRKIPKLEELLSKRDFTGAITLL...


In [6]:
# Extract sequences from the DataFrame into a list (corpus)
corpus = list(df['seq'])

# Print the first 10 sequences from the corpus
print(corpus[:10])

# Print the total number of sequences in the corpus
print(len(corpus))


['MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYEEKYKTFNKYWCRQPCLPIWHEMVETGGSEGVVRSDQVIITDHPGDLTFTVTLENLTADDAGKYRCGIATILQEDGLSGFLPDPFFQVQVLVSSASSTENSVKTPASPTRPSQCQGSLPSSTCFLLLPLLKVPLLLSILGAILWVNRPWRTPWTES', 'MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKSRSSSLTLPSSRDPQPLRIQSCYLGDHISDGAWDPEGEGMRGGSRALAAVREATQRIQAVLAVQGPLLLSRDPAQYCHAVWGDPDSPNYHRCSLLNPGYKGESCLGAKIPDTHLRGYALWPEQGPPQLVQPDGPGVQNTDFLLYVRVAHTSKCHQETVSLCCPGWSTAAQSQLTAALTSWAQRRGFVMLPRLCLKLLGSSNLPTLASQSIRITGPSVIAYAACCQLDSEDRPLAGTIVYCAQHLTSPSLSHSDIVMATLHELLHALGFSGQLFKKWRDCPSGFSVRENCSTRQLVTRQDEWGQLLLTTPAVSLSLAKHLGVSGASLGVPLEEEEGLLSSHWEARLLQGSLMTATFDGAQRTRLDPITLAAFKDSGWYQVNHSAAEELLWGQGSGPEFGLVTTCGTGSSDFFCTGSGLGCHYLHLDKGSCSSDPMLEGCRMYKPLANGSECWKKENGFPAGVDNPHGEIYHPQSRCFFANLTSQLLPGDKPRHPSLTPHLKEAELMGRCYLHQCTGRGAYKVQVEGSPWVPCLPGKVIQIPGYYGLLFCPRGRLCQTNEDINAVTSPPVSLSTPDPLFQLSLELAGPPGHSLGKEQQEGLAEAVLEALASKGGTGRCYFHGPSITTSLVFTVHMWKSPGCQGPSVATLHKALTLTLQKKPLEVYHGGANFTTQPSKLLVTSDHNPSMTHLRLSMGLCLMLLILVGVMGTTAYQKRATLPVRPSASYHSPELHSTRVPVRGIREV', 'M

In [7]:
# Define base mapping dictionaries for encoding and decoding
__mapping = {"A": 8, "C": 4, "G": 2, "T": 1, "N": 15, "E": 0}
__rmapping = {8: "A", 4: "C", 2: "G", 1: "T", 15: "N", 0: "E"}

# Define the base size for encoding
base_size = 2**8

# Function to encode a DNA sequence into an integer
def encode_sequence(sequence):
    val = 0
    for i, base in enumerate(sequence):
        print(base)
        val += __mapping[base] * (2**8**i)
    return val

# Function to decode an integer back into a DNA sequence
def decode_sequence(val):
    import math

    sequence = ""
    n = math.floor(math.log(val) / math.log(base_size))

    while val > 0:
        next_layer = val % base_size**n
        sequence = str(__rmapping[int((val - next_layer) / base_size**n)]) + sequence
        n -= 1
        val = next_layer

    return sequence


# Downsample: A Larger n Results in a Longer Training Time

In [8]:
n=2500

# Remove _

In [9]:
# Function to remove null amino acids from sequences
def remove_null_AA(corpus_dna_new):
    null_AAs = ['_', '_', "_", "_", "_", "_", "_"]
    results = []
    print(len(corpus_dna_new))
    for text in corpus_dna_new:
        tmp = list(text)
        for null_AA in null_AAs:
            if null_AA in tmp:
                tmp.remove(null_AA)
        results.append("".join(tmp))
    
    return results


In [10]:
# Function to extract unique amino acids from sequences
def amin(corpus_dna_new):
    amino_acids = []
    for text in corpus_dna_new:
        for AA in list(text):
            amino_acids.append(AA)
    
    amino_acids = set(amino_acids)
    return list(amino_acids)


In [11]:
# Extract and display unique amino acids from the corpus
amino_acids = amin(corpus)
amino_acids


['N',
 'Y',
 'S',
 'V',
 'I',
 'H',
 'D',
 'F',
 'P',
 'K',
 'T',
 'W',
 'Q',
 'L',
 'G',
 'E',
 'R',
 'A',
 'C',
 'U',
 'M']

# data generation

In [12]:
# Function to convert amino acids to integers and generate training data
def data_out(amino_acids, corpus_dna_new):
    AA2int = {}

    # Create a mapping from amino acids to integers
    for i, AA in enumerate(amino_acids):
        AA2int[AA] = i

    sentences = []
    for sentence in corpus_dna_new:
        sentences.append(list(sentence))

    WINDOW_SIZE = 2

    data = []
    for sentence in sentences:
        for idx, AA in enumerate(sentence):
            for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1]:
                if neighbor != AA:
                    data.append([AA, neighbor])
    
    return AA2int, data


In [13]:
# Generate the amino acid to integer mapping and training data
AA2int, data = data_out(amino_acids, corpus)

# Display the AA to integer mapping and training data
print(AA2int)
print(data)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [14]:
# Function to convert data into a pandas DataFrame
def pandify(data):
    df = pd.DataFrame(data, columns=['input', 'label'])
    return df


In [15]:
# Convert the data into a pandas DataFrame
df = pandify(data)

# Downsample the DataFrame to the first 'n' rows
df_downsampled = df.head(n)

# Display the downsampled DataFrame
df_downsampled


,input,label
0,M,T
1,M,Q
2,T,M
3,T,Q
4,T,R
...,...,...
2495,A,N
2496,A,G
2497,N,L
2498,N,A


# Define Tensorflow Graph

In [16]:
# Function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index, amino_acids):
    ONE_HOT_DIM = len(amino_acids)
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding


In [17]:
# Function to define the computational graph for training the model
def define_graph(AA2int, amino_acids, df):
    ONE_HOT_DIM = len(amino_acids)
    X = []  # input amino acid
    Y = []  # target amino acid

    for x, y in zip(df['input'], df['label']):
        X.append(to_one_hot_encoding(AA2int[x], amino_acids))
        Y.append(to_one_hot_encoding(AA2int[y], amino_acids))

    # Convert them to numpy arrays
    X_train = np.asarray(X)
    Y_train = np.asarray(Y)

    # Placeholders for X_train and Y_train
    x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
    y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

    # AA embedding will be 3 dimension for 3D visualization
    EMBEDDING_DIM = 3

    # Hidden layer: which represents AA vector eventually
    W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([1]))  # Bias
    hidden_layer = tf.add(tf.matmul(x, W1), b1)

    # Output layer
    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
    b2 = tf.Variable(tf.random_normal([1]))
    prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_layer, W2), b2))

    # Loss function: cross entropy
    loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

    # Training operation
    train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    return X_train, Y_train, x, y_label, W1, b1, W2, b2, loss, train_op


In [18]:
# Define the computational graph using the downsampled DataFrame
X_train_downsampled, Y_train_downsampled, x_downsampled, y_label_downsampled, W1_downsampled, b1_downsampled, W2_downsampled, b2_downsampled, loss_downsampled, train_op_downsampled = define_graph(AA2int, amino_acids, df_downsampled)

# Display the shapes of the training data arrays
print(f"X_train_downsampled shape: {X_train_downsampled.shape}")
print(f"Y_train_downsampled shape: {Y_train_downsampled.shape}")




X_train_downsampled shape: (2500, 21)
Y_train_downsampled shape: (2500, 21)


# Downsampled

## Train

In [19]:
# Initialize and run the TensorFlow session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# Training loop
iteration = 20000
for i in range(iteration):
    # Input is X_train which is one hot encoded AA
    # Label is Y_train which is one hot encoded neighbor AA
    sess.run(train_op_downsampled, feed_dict={x_downsampled: X_train_downsampled, y_label_downsampled: Y_train_downsampled})
    if i % 3000 == 0:
        loss_value = sess.run(loss_downsampled, feed_dict={x_downsampled: X_train_downsampled, y_label_downsampled: Y_train_downsampled})
        print(f'Iteration {i}, loss: {loss_value}')


Iteration 0, loss: 4.169535160064697
Iteration 3000, loss: 2.8225080966949463
Iteration 6000, loss: 2.8122880458831787
Iteration 9000, loss: 2.8074028491973877
Iteration 12000, loss: 2.8039770126342773
Iteration 15000, loss: 2.8012959957122803
Iteration 18000, loss: 2.7991864681243896


In [20]:
# Now the hidden layer (W1 + b1) is actually the AA lookup table
vectors_downsampled = sess.run(W1_downsampled + b1_downsampled)

# Optionally, print the vectors
# print(vectors_downsampled)


## AA vector in table

In [21]:
# Create a DataFrame with the amino acid vectors
w2v_df_downsampled = pd.DataFrame(vectors_downsampled, columns=['x1', 'x2', 'x3'])
w2v_df_downsampled['AA'] = amino_acids
w2v_df_downsampled = w2v_df_downsampled[['AA', 'x1', 'x2', 'x3']]

# Optionally, display the DataFrame
# w2v_df_downsampled


In [22]:
# Drop rows where the amino acid is "_"
w2v_df_downsampled.drop(w2v_df_downsampled[w2v_df_downsampled['AA'] == "_"].index, inplace=True)

# Display the cleaned DataFrame
w2v_df_downsampled


,AA,x1,x2,x3
0,N,2.572642,2.973097,2.579216
1,Y,2.594833,2.204943,2.352434
2,S,3.955608,3.229179,2.803545
3,V,3.639261,3.278837,2.874500
4,I,3.143257,2.535411,2.050777
5,H,3.229013,3.280474,2.692389
6,D,2.876619,2.963700,3.162906
7,F,3.366975,2.509836,2.098433
8,P,3.202193,2.803076,2.089284
9,K,3.018095,3.635763,3.182798


In [23]:
# Create a list of amino acids from the cleaned DataFrame
AA_lst_downsampled = list(w2v_df_downsampled['AA'])


In [24]:
# Define a color mapping for amino acids
color_mp = {
    'D': 'b', 'E': 'b',   # Blue for acidic amino acids
    'R': 'r', 'K': 'r', 'H': 'r',  # Red for basic amino acids
    'N': 'y', 'Q': 'y', 'S': 'y', 'T': 'y', 'Y': 'y',  # Yellow for polar uncharged amino acids
    'A': 'g', 'V': 'g', 'L': 'g', 'I': 'g', 'P': 'g', 'F': 'g', 'M': 'g', 'W': 'g', 'C': 'g', 'G': 'g'  # Green for nonpolar amino acids
}


In [25]:
# Generate a list of color codes for each amino acid in the list
color_code_downsampled = []
for i, elt in enumerate(AA_lst_downsampled):
    color_code_downsampled.append(color_mp.get(elt, 'w'))  # Default to 'w' (white) if amino acid not found in color_mp
AA_lst_downsampled

['N',
 'Y',
 'S',
 'V',
 'I',
 'H',
 'D',
 'F',
 'P',
 'K',
 'T',
 'W',
 'Q',
 'L',
 'G',
 'E',
 'R',
 'A',
 'C',
 'U',
 'M']

In [26]:
# Create a zip object with amino acid labels and their corresponding vectors
z_data_downsampled = zip(w2v_df_downsampled['AA'], w2v_df_downsampled['x1'], w2v_df_downsampled['x2'])

# Convert to a list if you need to view or iterate multiple times
z_data_downsampled = list(z_data_downsampled)

# Display the zipped data
z_data_downsampled


[('N', 2.572641611099243, 2.973097085952759),
 ('Y', 2.5948328971862793, 2.2049434185028076),
 ('S', 3.9556078910827637, 3.2291789054870605),
 ('V', 3.639260768890381, 3.2788374423980713),
 ('I', 3.143256902694702, 2.5354106426239014),
 ('H', 3.2290127277374268, 3.2804741859436035),
 ('D', 2.8766191005706787, 2.963700294494629),
 ('F', 3.3669745922088623, 2.509835720062256),
 ('P', 3.202192544937134, 2.8030762672424316),
 ('K', 3.018094778060913, 3.635763168334961),
 ('T', 2.9733026027679443, 3.224973678588867),
 ('W', 3.3008742332458496, 2.828317403793335),
 ('Q', 3.9262006282806396, 4.125444412231445),
 ('L', 3.1335887908935547, 2.8816568851470947),
 ('G', 2.8756744861602783, 3.73643159866333),
 ('E', 2.9324893951416016, 2.142150640487671),
 ('R', 3.298732280731201, 3.138536214828491),
 ('A', 3.057405710220337, 2.8126981258392334),
 ('C', 2.9260213375091553, 2.7240359783172607),
 ('U', 1.9274849891662598, 3.819521427154541),
 ('M', 2.2797818183898926, 2.2697367668151855)]

In [27]:
# Iterate through the zipped data and print the amino acids and their corresponding vectors
for (AA_downsampled, x1_downsampled, x2_downsampled) in z_data_downsampled:
    print(AA_downsampled, x1_downsampled, x2_downsampled)


N 2.572641611099243 2.973097085952759
Y 2.5948328971862793 2.2049434185028076
S 3.9556078910827637 3.2291789054870605
V 3.639260768890381 3.2788374423980713
I 3.143256902694702 2.5354106426239014
H 3.2290127277374268 3.2804741859436035
D 2.8766191005706787 2.963700294494629
F 3.3669745922088623 2.509835720062256
P 3.202192544937134 2.8030762672424316
K 3.018094778060913 3.635763168334961
T 2.9733026027679443 3.224973678588867
W 3.3008742332458496 2.828317403793335
Q 3.9262006282806396 4.125444412231445
L 3.1335887908935547 2.8816568851470947
G 2.8756744861602783 3.73643159866333
E 2.9324893951416016 2.142150640487671
R 3.298732280731201 3.138536214828491
A 3.057405710220337 2.8126981258392334
C 2.9260213375091553 2.7240359783172607
U 1.9274849891662598 3.819521427154541
M 2.2797818183898926 2.2697367668151855


### 3D chart

In [38]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')

for i in range(len(w2v_df_downsampled)):
    sc = ax.scatter(w2v_df_downsampled['x1'][i], w2v_df_downsampled['x2'][i], w2v_df_downsampled['x3'][i], c=color_code_downsampled[i], marker=r"$ {} $".format(AA_lst_downsampled[i]), s=100)
    
# for i in range(len(w2v_df_downsampled)):
#     sc = ax.scatter(w2v_df_downsampled['x1'][i], w2v_df_downsampled['x2'][i], w2v_df_downsampled['x3'][i], c=color_code_downsampled[i], marker=r"$ {} $".format(AA_lst_downsampled[i]), s=100, cmap="Spectral")    

# plt.colorbar(sc)
plt.show()

TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'